## Imports ##

In [2]:
import numpy as np
import scipy
import h5py
import rqcopt as oc
import matplotlib.pyplot as plt
from scipy import optimize
import ctypes

## Import C dynamic libraries ##

This code block is used to run rqcopt faster using the binaries compiled from the C code. You have to compile the dynamic library by yourself. If you cannot access the C code, you can replace the function <strong>optimize_brickwall_circuit</strong> later by the one from the rqcopt library in Python. That function, however, is significantly slower.

In [3]:
### CHANGE PATH HERE
rqcopt_dynlib_path = "rqcopt_lib.dylib"
rqcopt_lib = ctypes.CDLL(rqcopt_dynlib_path)
print(type(rqcopt_lib))

# Define the structure for truncated_cg_params
class TruncatedCGParams(ctypes.Structure):
    _fields_ = [("maxiter", ctypes.c_int),
                ("abstol", ctypes.c_double),
                ("reltol", ctypes.c_double)]

# Define the structure for rtr_params
class RTRParams(ctypes.Structure):
    _fields_ = [("tcg_params", TruncatedCGParams),
                ("rho_trust", ctypes.c_double),
                ("radius_init", ctypes.c_double),
                ("maxradius", ctypes.c_double),
                ("g_func", ctypes.c_void_p),
                ("g_data", ctypes.c_void_p),
                ("g_iter", ctypes.POINTER(ctypes.c_double))]

# Define the function signature for target_func
target_func = ctypes.CFUNCTYPE(ctypes.c_double, ctypes.POINTER(ctypes.c_double), ctypes.c_void_p)

# Define the optimize_brickwall_circuit function signature
optimize_brickwall_circuit = rqcopt_lib.optimize_brickwall_circuit_compact
optimize_brickwall_circuit.restype = None
optimize_brickwall_circuit.argtypes = [
    ctypes.c_void_p,                                  # udata
    ctypes.POINTER(ctypes.c_double * 32),             # Vlist_start
    ctypes.c_int,                                     # nlayers
    ctypes.c_int,                                     # L
    ctypes.POINTER(ctypes.POINTER(ctypes.c_int)),     # perms
    ctypes.c_int,                                     # niter
    ctypes.POINTER(ctypes.c_double),                  # f_iter
    ctypes.POINTER(ctypes.c_double * 32)              # Vlist_opt
]

<class 'ctypes.CDLL'>


## Function definitions ##

In [35]:
def trotterized_time_evolution(L: int, hloc, perm_set, method: oc.SplittingMethod, dt: float, nsteps: int):
    """
    Compute the numeric ODE flow operator of the quantum time evolution
    based on the provided splitting method.
    """
    Vlist = []
    perms = []
    for i, c in zip(method.indices, method.coeffs):
        Vlist.append(scipy.linalg.expm(-1j*c*dt*hloc[i]))
        perms.append(perm_set[i])
    V = oc.brickwall_unitary(Vlist, L, perms)
    return np.linalg.matrix_power(V, nsteps)

def construct_fermi_hubbard_local_kinetic_term(J):
    return -J * np.array([[0., 0., 0., 0.], [0., 0., 1., 0.], [0., 1., 0., 0.], [0., 0., 0., 0.]]);

def construct_fermi_hubbard_local_interaction_term(U):
    n_spin = np.array([[0., 0.], [0., 1.]])
    return U * np.kron(n_spin, n_spin);

def acting_two_qubit_gate(U, idx1, idx2, total_qubits):
    if idx2 < idx1:
        U = np.reshape(U, (2,2,2,2))
        U = np.transpose(U, (1,0, 3,2))
        U = np.reshape(U, (4,4))
        idx_temp = idx1
        idx1 = idx2
        idx2 = idx_temp
    mat = np.kron(U,np.eye(2**(total_qubits - 2)))
    print(mat.shape)
    mat = np.reshape(mat, (2,2,2**idx1, 2**(idx2 - idx1 - 1), 2**(total_qubits - idx2 - 1),2,2,2**idx1, 2**(idx2 - idx1 - 1), 2**(total_qubits - idx2 - 1)))
    mat = np.transpose(mat, (2,0,3,1,4, 7,5,8,6,9))
    mat = np.reshape(mat, (2**total_qubits, 2**total_qubits))
    return mat

def simulate_optimized_gate(vlist, perms, coeffs):
    n_qubits = np.shape(perms[0])[0]
    mat = np.eye(2**n_qubits)
    print(mat.shape)
    print(coeffs)
    for v_idx in coeffs:
        V = vlist[v_idx]
        print(V.shape)
        perm = perms[v_idx]
        for i in range(int(len(perm) / 2)):
            mat = acting_two_qubit_gate(V, perm[2*i], perm[2*i + 1], n_qubits) @ mat
    return mat

## Initial values ##

Globally default values

In [5]:
L = 4
J = 1.
U = 4
t = 1/8
strang = oc.SplittingMethod.suzuki(3, 3)
indices_start_n9, coeffs_start_n9 = oc.merge_layers(2*strang.indices, 2*strang.coeffs)
coeffs_start_n9 = [0.5*c for c in coeffs_start_n9]
method_start = oc.SplittingMethod(3, indices_start_n9, coeffs_start_n9, 2)

### DEFINE NUMER OF ITERATIONS HERE ###
nsteps = 50

horz_even_sites = np.array(range(2*L))
horz_odd_sites = np.roll(range(L*2), -1)
horz_odd_sites[[L - 1, L*2 -1]] = horz_odd_sites[[L*2 - 1, L -1]]
vert_sites = np.array([[i, i + L] for i in range(L)]).flatten()

perm_set = [
    horz_even_sites, 
    np.argsort(horz_odd_sites).tolist(), 
    np.argsort(vert_sites).tolist(),
    horz_even_sites, 
    np.argsort(horz_odd_sites).tolist(), 
    np.argsort(vert_sites).tolist()]

def get_perm_set(L):
    horz_even_sites = np.array(range(2*L))
    horz_odd_sites = np.roll(range(L*2), -1)
    horz_odd_sites[[L - 1, L*2 -1]] = horz_odd_sites[[L*2 - 1, L -1]]
    vert_sites = np.array([[i, i + L] for i in range(L)]).flatten()
    return [
        horz_even_sites, 
        np.argsort(horz_odd_sites).tolist(), 
        np.argsort(vert_sites).tolist(),
        horz_even_sites, 
        np.argsort(horz_odd_sites).tolist(), 
        np.argsort(vert_sites).tolist()]



bounds_pi = (0, 2 * np.pi)
bounds = [bounds_pi for i in range(4)]

## Reference Hamiltonian ##

In [6]:
def reference_hamiltonian(J, U, t, L):
    H = np.zeros((2**(2*L), 2**(2*L)))

    n_spin = np.eye(2)
    n_spin[0,0] = 0
    n_spin_big = np.kron(n_spin, np.kron(np.eye(2 ** (L - 1)), n_spin))
    for i in range (L):
        H += U * np.kron(np.eye(2 ** i), np.kron(n_spin_big, np.eye(2 ** (L - i - 1))))

    ### LOCAL KINETIC TERM
    hloc_horz = construct_fermi_hubbard_local_kinetic_term(J)
    hloc_vert = construct_fermi_hubbard_local_interaction_term(U)
    a_i = np.zeros((2,2))
    a_i1 = np.zeros((2,2))
    a_i[1,0] = 1
    a_i1[0,1] = 1
    for i in range (L - 1):
        H += np.kron(np.eye(2 ** i), np.kron(hloc_horz, np.eye(2**(2 * L - i - 2))))
    for i in range (L, 2*L - 1):
        H += np.kron(np.eye(2 ** i), np.kron(hloc_horz, np.eye(2**(2 * L - i - 2))))
    H += -J * np.kron(a_i1, np.kron(np.eye(2**(L - 2)), np.kron(a_i, np.eye(2**(L)))))
    H += -J * np.kron(a_i, np.kron(np.eye(2**(L - 2)), np.kron(a_i1, np.eye(2**(L)))))
    H += -J * np.kron(np.kron(np.eye(2**(L)), a_i1), np.kron(np.eye(2**(L - 2)), a_i))
    H += -J * np.kron(np.kron(np.eye(2**(L)), a_i), np.kron(np.eye(2**(L - 2)), a_i1))
    expiH = scipy.linalg.expm(-1j*H*t)

    print("H.shape:", H.shape)
    return H, expiH

## Optimization definitions ##

### Definitions using the C library ###

In [7]:
def optimize_fermi_hubbard_in_brickwall_layout_dylib(J, U, t, hloc_horz, hloc_vert, splitting, L):
    Vlist_start = []
    perms = []
    hloc = [hloc_horz, hloc_horz, hloc_vert]
    perm_set = get_perm_set(L)
    for i, c in zip(splitting.indices, splitting.coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*t*hloc[i]))
        perms.append(perm_set[i])

    Vlist_start = np.array(Vlist_start)
    perms = np.array(perms, dtype='int32')
    Vlist_opt = np.zeros_like(Vlist_start)
    f_iter = np.zeros(nsteps, dtype=np.float64)
    c_perems = (ctypes.POINTER(ctypes.c_int) * perms.shape[0])()
    for i, row in enumerate(perms):
        c_perems[i] = row.ctypes.data_as(ctypes.POINTER(ctypes.c_int))

    H, expiH = reference_hamiltonian(J, U, t, L)
    
    optimize_brickwall_circuit(
        expiH.ctypes.data_as(ctypes.POINTER(ctypes.c_double * 2)), 
        Vlist_start.ctypes.data_as(ctypes.POINTER(ctypes.c_double * 32)), 
        np.shape(perms)[0], 
        2 * L, 
        c_perems, 
        nsteps, 
        f_iter.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),  
        Vlist_opt.ctypes.data_as(ctypes.POINTER(ctypes.c_double * 32)))
    return Vlist_opt, f_iter

def optimize_fermi_hubbard_in_brickwall_layout_dylib_double(J, U, t, hloc_horz, hloc_vert, splitting, L):
    Vlist_start = []
    perms = []
    hloc = [hloc_horz, hloc_horz, hloc_vert, np.eye(4), np.eye(4), np.eye(4)]
    perm_set = get_perm_set(L)
    for i, c in zip(splitting.indices, splitting.coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*t*hloc[i]))
        perms.append(perm_set[i])

    Vlist_start = np.array(Vlist_start)
    perms = np.array(perms, dtype='int32')
    Vlist_opt = np.zeros_like(Vlist_start)
    f_iter = np.zeros(nsteps, dtype=np.float64)
    c_perems = (ctypes.POINTER(ctypes.c_int) * perms.shape[0])()
    for i, row in enumerate(perms):
        c_perems[i] = row.ctypes.data_as(ctypes.POINTER(ctypes.c_int))

    H, expiH = reference_hamiltonian(J, U, t, L)
    
    optimize_brickwall_circuit(
        expiH.ctypes.data_as(ctypes.POINTER(ctypes.c_double * 2)), 
        Vlist_start.ctypes.data_as(ctypes.POINTER(ctypes.c_double * 32)), 
        np.shape(perms)[0], 
        2 * L, 
        c_perems, 
        nsteps, 
        f_iter.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),  
        Vlist_opt.ctypes.data_as(ctypes.POINTER(ctypes.c_double * 32)))
    return Vlist_opt, f_iter

def optimize_fermi_hubbard_in_brickwall_layout_dylib_double_ref(J, U, t, hloc_horz, hloc_vert, splitting, L):
    Vlist_start = []
    perms = []
    hloc = [hloc_horz, hloc_horz, hloc_vert, hloc_horz, hloc_horz, hloc_vert]
    perm_set = get_perm_set(L)
    for i, c in zip(splitting.indices, splitting.coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*t*hloc[i]))
        perms.append(perm_set[i])

    Vlist_start = np.array(Vlist_start)
    perms = np.array(perms, dtype='int32')
    Vlist_opt = np.zeros_like(Vlist_start)
    f_iter = np.zeros(nsteps, dtype=np.float64)
    c_perems = (ctypes.POINTER(ctypes.c_int) * perms.shape[0])()
    for i, row in enumerate(perms):
        c_perems[i] = row.ctypes.data_as(ctypes.POINTER(ctypes.c_int))

    H, expiH = reference_hamiltonian(J, U, t, L)
    
    optimize_brickwall_circuit(
        expiH.ctypes.data_as(ctypes.POINTER(ctypes.c_double * 2)), 
        Vlist_start.ctypes.data_as(ctypes.POINTER(ctypes.c_double * 32)), 
        np.shape(perms)[0], 
        2 * L, 
        c_perems, 
        nsteps, 
        f_iter.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),  
        Vlist_opt.ctypes.data_as(ctypes.POINTER(ctypes.c_double * 32)))
    return Vlist_opt, f_iter

def optimize_fermi_hubbard_in_brickwall_layout_dylib_bootstrap(J, U, t, Vlist_start_original, splitting_start, L):
    assert len(Vlist_start_original) == len(splitting_start.coeffs)
    Vlist_start_original = np.concatenate(np.array([[np.identity(4), np.identity(4)], Vlist_start_original, [np.identity(4), np.identity(4)]]), axis = 0)
    perms = [horz_even_sites, horz_odd_sites]
    perm_set = get_perm_set(L)

    Vlist_start = Vlist_start_original
    for i, c in zip(splitting_start.indices, splitting_start.coeffs):
        perms.append(perm_set[i])
    
    perms.append(horz_odd_sites)
    perms.append(horz_even_sites)

    assert len(splitting_start.coeffs) == len(Vlist_start) - 4
    assert len(splitting_start.coeffs) == len(perms) - 4

    Vlist_start = np.array(Vlist_start)
    perms = np.array(perms, dtype='int32')
    Vlist_opt = np.zeros_like(Vlist_start)
    f_iter = np.zeros(nsteps, dtype=np.float64)
    c_perems = (ctypes.POINTER(ctypes.c_int) * perms.shape[0])()
    for i, row in enumerate(perms):
        c_perems[i] = row.ctypes.data_as(ctypes.POINTER(ctypes.c_int))

    H, expiH = reference_hamiltonian(J, U, t, L)
    
    optimize_brickwall_circuit(
        expiH.ctypes.data_as(ctypes.POINTER(ctypes.c_double * 2)), 
        Vlist_start.ctypes.data_as(ctypes.POINTER(ctypes.c_double * 32)), 
        np.shape(perms)[0], 
        2 * L, 
        c_perems, 
        nsteps, 
        f_iter.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),  
        Vlist_opt.ctypes.data_as(ctypes.POINTER(ctypes.c_double * 32)))
    return Vlist_opt, f_iter

### Example definition using the Python library ###

In [8]:
def optimize_fermi_hubbard_in_brickwall_layout():
    Vlist_start = []
    perms = []
    hloc = [hloc_horz, hloc_horz, hloc_vert]
    for i, c in zip(method_start.indices, method_start.coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*t*hloc[i]))
        perms.append(perm_set[i])
    return oc.optimize_brickwall_circuit(2 * L, expiH, Vlist_start, perms, niter=nsteps)

## Optimization ##

### Trivial set-up ###

In [ ]:
hamiltonian_params = [(1,4,1/8), (1,4,1/4), (1,4,1/2), (1,8,1/8), (1,8,1/4), (1,8,1/2)]
H = 1 / np.sqrt(2) * np.array([[1, 1], [1, -1]])
### SPLTTING METHOD CAN BE CHANGE HERE ###
splitting = oc.SplittingMethod.suzuki(3,2)

results_err = []
results_vlist = []
for i in range(len(hamiltonian_params)):
    (J, U, t) = hamiltonian_params[i]

    ### DEFINE v_kin AND v_inter HERE ###
    v_kin = construct_fermi_hubbard_local_kinetic_term(J)
    v_inter = construct_fermi_hubbard_local_interaction_term(U)
    # v_kin = np.kron(H, H)
    # v_inter = np.kron(H, H)
    # v_kin = np.zeros((4,4))
    # v_inter = np.zeros((4,4))
    vlist, err = optimize_fermi_hubbard_in_brickwall_layout_dylib(J, U, t, v_kin, v_inter, splitting, 4)
    results_err.append(err)
    results_vlist.append(vlist)


### Trivial set-up ###

In [ ]:
hamiltonian_params = [(1,4,1/8), (1,4,1/4), (1,4,1/2), (1,8,1/8), (1,8,1/4), (1,8,1/2)]
H = 1 / np.sqrt(2) * np.array([[1, 1], [1, -1]])
### SPLTTING METHOD CAN BE CHANGE HERE ###
splitting = oc.SplittingMethod.suzuki(5,1)

results_err = []
results_vlist = []
for i in range(len(hamiltonian_params)):
    (J, U, t) = hamiltonian_params[i]
    v_kin = construct_fermi_hubbard_local_kinetic_term(J)
    v_inter = construct_fermi_hubbard_local_interaction_term(U)
    vlist, err = optimize_fermi_hubbard_in_brickwall_layout_dylib_double(J, U, t, v_kin, v_inter, splitting, 4)
    results_err.append(err)
    results_vlist.append(vlist)

### Comparing different lengths of the system ###

In [ ]:
hamiltonian_params = (1,4,1/8)
## SPLTTING METHOD CAN BE CHANGE HERE #
splitting = oc.SplittingMethod.suzuki(3,1)
print(splitting.num_terms)
print(splitting.num_layers)
print(splitting.coeffs)


results_err = []
results_vlist = []
Ls = [2,4,6]
for L in Ls:
    (J, U, t) = hamiltonian_params
    v_kin = construct_fermi_hubbard_local_kinetic_term(J)
    v_inter = construct_fermi_hubbard_local_interaction_term(U)
    vlist, err = optimize_fermi_hubbard_in_brickwall_layout_dylib(J, U, t, v_kin, v_inter, splitting, L)
    results_err.append(err)
    results_vlist.append(vlist)

### Bootstrapping ###

In [ ]:
hamiltonian_params = [(1,4,1/8), (1,4,1/4), (1,4,1/2), (1,8,1/8), (1,8,1/4), (1,8,1/2)]
results_err = []
results_vlist = []
with h5py.File(f"basic_4th.hdf5", "r") as f:
    vlist_start = f["vlist"]
    for i in range(len(hamiltonian_params)):
        (J, U, t) = hamiltonian_params[i]
        vlist, err = optimize_fermi_hubbard_in_brickwall_layout_dylib_bootstrap(J, U, t, vlist_start[i], oc.SplittingMethod.suzuki(3,2), 4)
        results_err.append(err)
        results_vlist.append(vlist)

## Save and plot results ##

The following blocks can be used to save, load and plot the results.

In [37]:
def plot_opt_multiple(err_list, hamiltonian_params, title, filename):
    plt.xlabel("Iteration")
    plt.ylabel("Error")
    plt.title(title)
    markers = ['o', '*', '.', '1', 'x', '+', 's']
    for i in range(len(hamiltonian_params)):
        print(err_list[i][-1] + 2 ** (2 * L))
        plt.semilogy(range(len(err_list[i])), (err_list[i] + 2**(2*L))/256, marker=markers[i], color="C{}".format(i))
    plt.legend([f"({hamiltonian_params[i][0]}, {hamiltonian_params[i][1]}, {hamiltonian_params[i][2]})" for i in range(len(hamiltonian_params))], loc='upper right')
    plt.savefig(filename)

### Save results ###

In [36]:
with h5py.File("opt_results/combined6_4th_ref.hdf5", "w") as f:
    f.create_dataset("error", data=np.array(results_err))
    f.create_dataset("vlist", data=np.array(results_vlist))


### Show and safe plot from results_err ###

In [ ]:
plot_opt_multiple(results_err, hamiltonian_params, "Optimization progress, comparison to combined,\nVlist_start:=[U_kin, U_kin, U_int, U_kin, U_kin, U_int],\nSuzuki (4th order)", "plots/combined6_4th_ref.pdf")


### Show and save plot from the loaded data ###

In [ ]:
with h5py.File(f"opt_results/basic_4th_bootstrap.hdf5", "r") as f:
    plot_opt_multiple(f["error"], hamiltonian_params, "Optimization progress\nVlist_start:=[kron(H,H),kron(H,H),kron(H,H)], Suzuki (2nd order)", "plots/name.pdf")


## Number operator ##

The following blocks can be use to check, whether the resulting circuit commutes ith the number operator

In [12]:
def number_operator(N):
    mat = np.eye(2**(2*N), dtype=complex)
    create_op = np.array([[0., 0.], [1., 0.]])
    annihilate_op = np.array([[0., 1.], [0., 0.]])
    for i in range(2 * N):
        mat = np.kron(np.eye(int(2**(i))), np.kron(create_op @ annihilate_op, np.eye(int(2**(2 * N-i-1))))) @ mat
        
    return mat

In [ ]:
np.linalg.norm(number_operator(L) @ reference_hamiltonian(1, 4, 0.125, 4) - reference_hamiltonian(1, 4, 0.125, 4) @ number_operator(L))

H.shape: (256, 256)
H.shape: (256, 256)


0.0

In [ ]:
Ux = simulate_optimized_gate(results_vlist[0], perm_set, splitting.indices)
N = number_operator(int(L))
H = reference_hamiltonian(1, 4, 0.5, 4)
np.linalg.norm(Ux @ N - N @ Ux, ord=2)